<a href="https://colab.research.google.com/github/JamshedAli18/Data-Preprocessing-Scikit-learn/blob/main/Pipe_Lines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [3]:
from sklearn.datasets import fetch_openml

# Load Titanic dataset
titanic = fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,None,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


In [6]:
df.drop(['name','ticket','cabin','boat','body','home.dest'],axis='columns',inplace=True)

In [7]:
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1.0,1,female,29.0000,0.0,0.0,211.3375,S
1,1.0,1,male,0.9167,1.0,2.0,151.5500,S
2,1.0,0,female,2.0000,1.0,2.0,151.5500,S
3,1.0,0,male,30.0000,1.0,2.0,151.5500,S
4,1.0,0,female,25.0000,1.0,2.0,151.5500,S


In [10]:
X = df.drop('survived',axis='columns')
y = df['survived']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked
772,3.0,male,17.0,0.0,0.0,7.8958,S
543,2.0,male,36.0,0.0,0.0,10.5000,S
289,1.0,female,18.0,0.0,2.0,79.6500,S
10,1.0,male,47.0,1.0,0.0,227.5250,C
147,1.0,male,NaN,0.0,0.0,42.4000,S


In [19]:
# Imputation
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [20]:
# One hot Encode
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [21]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1, 6])])

In [23]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [30]:
# trf4 = SelectKBest(score_func=chi2,k=5)

In [25]:
from sklearn.tree import DecisionTreeClassifier
trf5 = DecisionTreeClassifier()

In [31]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    # ('trf4',trf4),
    ('trf5',trf5),
])

In [33]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

In [34]:
pipe.score(X_test,y_test)

0.6030534351145038

# EXPORTING PIPELINE

In [35]:
import pickle

In [36]:
pickle.dump(pipe,open('pipeline.pkl','wb'))

In [38]:
X_test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
1148,3.0,male,35.0,0.0,0.0,7.1250,S
1049,3.0,male,20.0,1.0,1.0,15.7417,C
982,3.0,male,NaN,0.0,0.0,7.8958,S
808,3.0,male,NaN,0.0,0.0,8.0500,S
1195,3.0,male,NaN,0.0,0.0,7.7500,Q


In [39]:
y_test.head()

1148    0
1049    1
982     0
808     0
1195    0
Name: survived, dtype: category
Categories (2, object): ['0', '1']